In [1]:
# Installation Long Shhort-Term Memory with PyTorch + Lightning
import torch
import torch.nn as nn
import torch.nn.functional as f
from torch.optim import Adam


In [3]:
print(torch.__version__)

2.5.1+cpu


In [5]:
import lightning as l
from torch.utils.data import TensorDataset, DataLoader

In [7]:
# LSTMByHand inheritance l.LightningModule of Pytorch Lightning
class LSTMByHand(l.LightningModule):
    
    # Create and initialize all Weight and Bias tensors that we need to implement an LSTM unit
    def __init__(self):
        # Call the initialization method for the parent class (LightningModule)
        super().__init__()
        # Use Normal Distribution to randomly select an initialization value for each Weight but Bias = 0
        mean = torch.tensor(0.0)
        std = torch.tensor(1.0)
        
        self.wlr1 = nn.Parameter(torch.normal(mean=mean, std=std), requires_grad=True)
        self.wlr2 = nn.Parameter(torch.normal(mean=mean, std=std), requires_grad=True)
        self.blr1 = nn.Parameter(torch.tensor(0.), requires_grad=True)

        self.wpr1 = nn.Parameter(torch.normal(mean=mean, std=std), requires_grad=True)
        self.wpr2 = nn.Parameter(torch.normal(mean=mean, std=std), requires_grad=True)
        self.bpr1 = nn.Parameter(torch.tensor(0.), requires_grad=True)

        self.wp1 = nn.Parameter(torch.normal(mean=mean, std=std), requires_grad=True)
        self.wp2 = nn.Parameter(torch.normal(mean=mean, std=std), requires_grad=True)
        self.bp1 = nn.Parameter(torch.tensor(0.), requires_grad=True)

        self.wo1 = nn.Parameter(torch.normal(mean=mean, std=std), requires_grad=True)
        self.wo2 = nn.Parameter(torch.normal(mean=mean, std=std), requires_grad=True)
        self.bo1 = nn.Parameter(torch.tensor(0.), requires_grad=True)
        
    # Do LSTM math
    def lstm_unit(self, input_value, long_memory, short_memory):
        # Forget Gate
        long_rememeber_percent = torch.sigmoid((short_memory * self.wlr1) + (input_value * self.wlr2) + self.blr1)
        forget_long_memory = long_memory * long_rememeber_percent
        
        # Input Gate
        potential_remember_percent = torch.sigmoid((short_memory * self.wpr1) + (input_value * self.wpr2) + self.bpr1)
        potential_memory = torch.tanh((short_memory * self.wp1) + (short_memory * self.wp2) + self.bp1)
        updated_long_memory = forget_long_memory + (potential_memory * potential_remember_percent)

        # Output Gate
        output_percent = torch.sigmoid((short_memory * self.wo1) + (input_value * self.wo2) + self.bo1)
        updated_short_memory = torch.tanh(updated_long_memory) * output_percent

        # Pass LTM and STM to another Unit
        return ([updated_long_memory, updated_short_memory])
        
    # Makes a forward pass through the unrolled LSTM unit
    def forward(sefl, input):
        # # Input is a sequence data 
        # long_memory = 0
        # short_memory = 0

        # for t in range(input.size(0)): # input.size(0) will return seq_length
        #     input_t = input[t] # Get data in t time
        #     long_memory, short_memory = sefl.lstm_unit(input_t, long_memory, short_memory)

        long_memory = 0
        short_memory = 0
        day1 = input[0]
        day2 = input[1]
        day3 = input[2]
        day4 = input[3]

        long_memory, short_memory = sefl.lstm_unit(day1, long_memory, short_memory)
        long_memory, short_memory = sefl.lstm_unit(day2, long_memory, short_memory)
        long_memory, short_memory = sefl.lstm_unit(day3, long_memory, short_memory)
        long_memory, short_memory = sefl.lstm_unit(day4, long_memory, short_memory)

        return short_memory

    # Configure the optimizer we want to use:
    def configure_optimizers(self):
        return Adam(self.parameters(), lr= 0.1)
        
    # Calculate the loss and log trainning progress
    def training_step(self, batch, batch_idx):
        input_i, label_i = batch
        output_i = self.forward(input_i[0])
    
        # Tính toán loss
        loss = (output_i - label_i) ** 2
    
        # Log giá trị trung bình của loss (scalar)
        self.log("train_loss", loss)
    
        # Nếu `label_i` có nhiều phần tử, cần duyệt qua từng phần tử
        for i, label in enumerate(label_i):
            if label.item() == 0:
                self.log(f'out_0_batch_{i}', output_i)
            else:
                self.log(f'out_1_batch_{i}', output_i)
    
        # Trả về giá trị loss trung bình
        return loss        
        

In [9]:
model = LSTMByHand()
print("\nNow let's compare the observed and predicted values...")
print("Company A: Observed = 0, Predicted = ", model(torch.tensor([0., 0.5, 0.25, 1.])).detach())


Now let's compare the observed and predicted values...
Company A: Observed = 0, Predicted =  tensor(0.)


In [11]:
print("\nNow let's compare the observed and predicted values...")
print("Company B: Observed = 1, Predicted = ", model(torch.tensor([1., 0.5, 0.25, 1.])).detach())


Now let's compare the observed and predicted values...
Company B: Observed = 1, Predicted =  tensor(0.)


In [13]:
inputs = torch.tensor([[0., 0.5, 0.25, 1.], [1., 0.5, 0.25, 1.]])
labels = torch.tensor([0., 1.])

In [15]:
# Tạo Dataset và DataLoader
dataset = TensorDataset(inputs, labels)
dataloader = DataLoader(dataset)

# Tối ưu CPU
torch.set_num_threads(4)

In [17]:
trainer = l.Trainer(max_epochs=2000)
print("Starting training...")
trainer.fit(model, train_dataloaders=dataloader)
print("Training finished!")

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Starting training...



  | Name         | Type | Params | Mode
---------------------------------------------
  | other params | n/a  | 12     | n/a 
---------------------------------------------
12        Trainable params
0         Non-trainable params
12        Total params
0.000     Total estimated model params size (MB)
0         Modules in train mode
0         Modules in eval mode
D:\anaconda\src\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
D:\anaconda\src\Lib\site-packages\lightning\pytorch\loops\fit_loop.py:298: The number of training batches (2) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |                                                                                      | 0/? [00:00<…

`Trainer.fit` stopped: `max_epochs=2000` reached.


Training finished!


In [19]:
print("\nNow let's compare the observed and predicted values...")
print("Company A: Observed = 0, Predicted = ", model(torch.tensor([0., 0.5, 0.25, 1.])).detach())


Now let's compare the observed and predicted values...
Company A: Observed = 0, Predicted =  tensor(0.0669)


In [21]:
print("\nNow let's compare the observed and predicted values...")
print("Company B: Observed = 1, Predicted = ", model(torch.tensor([1., 0.5, 0.25, 1.])).detach())


Now let's compare the observed and predicted values...
Company B: Observed = 1, Predicted =  tensor(0.9721)


In [23]:
path_to_best_checkpoint = trainer.checkpoint_callback.best_model_path

In [25]:
trainer = l.Trainer(max_epochs=3000)
trainer.fit(model, train_dataloaders=dataloader, ckpt_path=path_to_best_checkpoint)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Restoring states from the checkpoint path at D:\anaconda\srcCode\ExampleStockPrediction\lightning_logs\version_48\checkpoints\epoch=1999-step=4000.ckpt
D:\anaconda\src\Lib\site-packages\lightning\pytorch\callbacks\model_checkpoint.py:362: The dirpath has changed from 'D:\\anaconda\\srcCode\\ExampleStockPrediction\\lightning_logs\\version_48\\checkpoints' to 'D:\\anaconda\\srcCode\\ExampleStockPrediction\\lightning_logs\\version_49\\checkpoints', therefore `best_model_score`, `kth_best_model_path`, `kth_value`, `last_model_path` and `best_k_models` won't be reloaded. Only `best_model_path` will be reloaded.

  | Name         | Type | Params | Mode
---------------------------------------------
  | other params | n/a  | 12     | n/a 
---------------------------------------------
12        Trainable params
0         Non-trainable params
12        Total params
0.000     Total estim

Training: |                                                                                      | 0/? [00:00<…

`Trainer.fit` stopped: `max_epochs=3000` reached.


In [27]:
print("\nNow let's compare the observed and predicted values...")  
print("Company A: Observed = 0, Predicted = ", model(torch.tensor([0., 0.5, 0.25, 1.])).detach())


Now let's compare the observed and predicted values...
Company A: Observed = 0, Predicted =  tensor(0.0768)


In [29]:
print("\nNow let's compare the observed and predicted values...")
print("Company B: Observed = 1, Predicted = ", model(torch.tensor([1., 0.5, 0.25, 1.])).detach())


Now let's compare the observed and predicted values...
Company B: Observed = 1, Predicted =  tensor(0.9702)


In [31]:
path_to_best_checkpoint = trainer.checkpoint_callback.best_model_path
trainer = l.Trainer(max_epochs=5000)
trainer.fit(model, train_dataloaders=dataloader, ckpt_path=path_to_best_checkpoint)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Restoring states from the checkpoint path at D:\anaconda\srcCode\ExampleStockPrediction\lightning_logs\version_49\checkpoints\epoch=2999-step=6000.ckpt
D:\anaconda\src\Lib\site-packages\lightning\pytorch\callbacks\model_checkpoint.py:362: The dirpath has changed from 'D:\\anaconda\\srcCode\\ExampleStockPrediction\\lightning_logs\\version_49\\checkpoints' to 'D:\\anaconda\\srcCode\\ExampleStockPrediction\\lightning_logs\\version_50\\checkpoints', therefore `best_model_score`, `kth_best_model_path`, `kth_value`, `last_model_path` and `best_k_models` won't be reloaded. Only `best_model_path` will be reloaded.

  | Name         | Type | Params | Mode
---------------------------------------------
  | other params | n/a  | 12     | n/a 
---------------------------------------------
12        Trainable params
0         Non-trainable params
12        Total params
0.000     Total estim

Training: |                                                                                      | 0/? [00:00<…

`Trainer.fit` stopped: `max_epochs=5000` reached.


In [33]:
print("\nNow let's compare the observed and predicted values...")  
print("Company A: Observed = 0, Predicted = ", model(torch.tensor([0., 0.5, 0.25, 1.])).detach())


Now let's compare the observed and predicted values...
Company A: Observed = 0, Predicted =  tensor(0.0421)


In [35]:
print("\nNow let's compare the observed and predicted values...")
print("Company B: Observed = 1, Predicted = ", model(torch.tensor([1., 0.5, 0.25, 1.])).detach())


Now let's compare the observed and predicted values...
Company B: Observed = 1, Predicted =  tensor(0.9703)


In [37]:
path_to_best_checkpoint = trainer.checkpoint_callback.best_model_path
trainer = l.Trainer(max_epochs=10000)
print("Starting training...")
trainer.fit(model, train_dataloaders=dataloader, ckpt_path=path_to_best_checkpoint)
print("Training finished!")

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Restoring states from the checkpoint path at D:\anaconda\srcCode\ExampleStockPrediction\lightning_logs\version_50\checkpoints\epoch=4999-step=10000.ckpt
D:\anaconda\src\Lib\site-packages\lightning\pytorch\callbacks\model_checkpoint.py:362: The dirpath has changed from 'D:\\anaconda\\srcCode\\ExampleStockPrediction\\lightning_logs\\version_50\\checkpoints' to 'D:\\anaconda\\srcCode\\ExampleStockPrediction\\lightning_logs\\version_51\\checkpoints', therefore `best_model_score`, `kth_best_model_path`, `kth_value`, `last_model_path` and `best_k_models` won't be reloaded. Only `best_model_path` will be reloaded.

  | Name         | Type | Params | Mode
---------------------------------------------
  | other params | n/a  | 12     | n/a 
---------------------------------------------
12        Trainable params
0         Non-trainable params
12        Total params
0.000     Total esti

Starting training...


Training: |                                                                                      | 0/? [00:00<…

`Trainer.fit` stopped: `max_epochs=10000` reached.


Training finished!


In [39]:
print("\nNow let's compare the observed and predicted values...")  
print("Company A: Observed = 0, Predicted = ", model(torch.tensor([0., 0.5, 0.25, 1.])).detach())
print("\nNow let's compare the observed and predicted values...")
print("Company B: Observed = 1, Predicted = ", model(torch.tensor([1., 0.5, 0.25, 1.])).detach())


Now let's compare the observed and predicted values...
Company A: Observed = 0, Predicted =  tensor(0.0019)

Now let's compare the observed and predicted values...
Company B: Observed = 1, Predicted =  tensor(0.9957)


In [42]:
path_to_best_checkpoint = trainer.checkpoint_callback.best_model_path
trainer = l.Trainer(max_epochs=2000)
print("Starting training...")
trainer.fit(model, train_dataloaders=dataloader, ckpt_path=path_to_best_checkpoint)
print("Training finished!")

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Restoring states from the checkpoint path at D:\anaconda\srcCode\ExampleStockPrediction\lightning_logs\version_45\checkpoints\epoch=9999-step=20000.ckpt
D:\anaconda\src\Lib\site-packages\lightning\pytorch\callbacks\model_checkpoint.py:362: The dirpath has changed from 'D:\\anaconda\\srcCode\\ExampleStockPrediction\\lightning_logs\\version_45\\checkpoints' to 'D:\\anaconda\\srcCode\\ExampleStockPrediction\\lightning_logs\\version_46\\checkpoints', therefore `best_model_score`, `kth_best_model_path`, `kth_value`, `last_model_path` and `best_k_models` won't be reloaded. Only `best_model_path` will be reloaded.

  | Name         | Type | Params | Mode
---------------------------------------------
  | other params | n/a  | 12     | n/a 
---------------------------------------------
12        Trainable params
0         Non-trainable params
12        Total params
0.000     Total esti

Starting training...


MisconfigurationException: You restored a checkpoint with current_epoch=9999, but you have set Trainer(max_epochs=2000).

NameError: name 'actual' is not defined